# I. US County Race Population

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report


path = 'DataMerge/Data/C-AnnualCountyPopulation/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [2]:

STATE_VACCINE = {}


In [3]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)


Enter the date of collecting data (Today's data in YYYY-MM-DD):


 2021-04-28


Hello, 2021-04-28


# II. API States

## [!] A. California* (Done)


https://covid19.ca.gov/vaccines/#California-vaccines-dashboard


https://data.ca.gov/dataset/covid-19-vaccine-progress-dashboard-data 


https://data.ca.gov/dataset/covid-19-vaccine-progress-dashboard-data/resource/c020ef6b-2116-4775-b11d-9df2875096ab # click here



Statewide COVID-19 Vaccines Administered By CountyCSV


Direct Vaccine Rate 

No Fully Rate information

In [4]:
'CountyVaccine/TMP/April28/California/covid19vaccinesbycounty (1).csv'

'CountyVaccine/TMP/April28/California/covid19vaccinesbycounty (1).csv'

In [6]:
import os
from datetime import datetime 
import numpy as np
import requests
import time
 
Date = Today_Date
    
while True:
    Path = input('Enter California Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)



# CalInfo: county total vaccine number each date
CalInfo = pd.read_csv(Path)
CalInfo['administered_date'] = pd.to_datetime(CalInfo['administered_date'])
CalInfo = CalInfo[CalInfo['administered_date'] == Date].reset_index(drop = True)


while len(CalInfo) == 0:
    print('There is no information for current Date:', Date)
    Date = input('Please Enter a new One:')
    CalInfo = pd.read_csv(Path)
    CalInfo = CalInfo[CalInfo['administered_date'] == Date].reset_index(drop = True)
    

print('We get California Information for Date:', Date)
CalInfo = CalInfo[['county', 'california_flag', 'cumulative_at_least_one_dose']]
CalInfo['Geographic Area Name'] = CalInfo['county'] + ', ' + CalInfo['california_flag']
print(CalInfo.shape)


CalInfo

Enter California Manually collected Data:   CountyVaccine/TMP/April28/California/covid19vaccinesbycounty (1).csv


File Exists!


Enter "Yes" if the file contains the information of the date you wanted:   Yes



The Path we have:
CountyVaccine/TMP/April28/California/covid19vaccinesbycounty (1).csv
There is no information for current Date: 2021-04-28


Please Enter a new One: 2021-04-27


We get California Information for Date: 2021-04-27
(62, 4)


,county,california_flag,cumulative_at_least_one_dose,Geographic Area Name
0,All CA Counties,NaN,18003849,NaN
1,All CA and Non-CA Counties,NaN,18370060,NaN
2,Outside California,Not in California,361949,"Outside California, Not in California"
3,Unknown,Not in California,4262,"Unknown, Not in California"
4,Alameda,California,918683,"Alameda, California"
...,...,...,...,...
57,Tulare,California,149721,"Tulare, California"
58,Tuolumne,California,21238,"Tuolumne, California"
59,Ventura,California,399353,"Ventura, California"
60,Yolo,California,105836,"Yolo, California"


In [7]:

def get_California_Raw_and_postprocess(US_COUNTY_POPULATION):
    state = 'California'
    ca_county_list = [i.lower().replace(' ','') for i in US_COUNTY_POPULATION[US_COUNTY_POPULATION['State'] == 'California']['County'].values]

    L = []
    error_list = []
    for i in ca_county_list:

        api_url = 'https://files.covid19.ca.gov/data/vaccine-equity/race-ethnicity/vaccines_by_race_ethnicity_{}.json'.format(i)

        resp = requests.get(api_url)# .to_json()
        data = resp.json() # resp

        try:

            d = {}
            d['County'] = data['meta']['REGION']
            d['Latest_Date'] = data['meta']['LATEST_ADMIN_DATE']
            for i in data['data']:
                d[i['CATEGORY']] = i['METRIC_VALUE']
            L.append(d)

        except:
            print(i)
            error_list.append(i)

    CA_County_Vaccine_Race = pd.DataFrame(L)
    
    return CA_County_Vaccine_Race





def standardize_California(CA_County_Vaccine_Race, US_COUNTY_POPULATION):
    
    # IL_County_Vaccine_Race = pd.DataFrame(L)
    state = 'California'

    new_L = []


    for idx, row in CA_County_Vaccine_Race.iterrows():
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = d['County']
        
        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        # new_d['Numb_White'] = np.nan
        new_d['Rate_White_V'] = d['White']

        # new_d['Numb_Black'] = np.nan
        new_d['Rate_Black_V'] = d['Black']

        # new_d['Numb_Asian'] = np.nan
        new_d['Rate_Asian_V'] = d['Asian American']

        # new_d['Numb_AIndA'] = np.nan
        new_d['Rate_AIndA_V'] = d['American Indian or Alaska Native (AI/AN)']

        # new_d['Numb_Other'] = np.nan
        new_d['Rate_Other_V'] = d['Other']

        new_d['Date'] = d['Latest_Date']

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)

    
    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')

        
        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return T
    

    

CA_County_Vaccine_Race = get_California_Raw_and_postprocess(US_COUNTY_POPULATION)

# CA_County_Vaccine_Race.to_csv('FinalResult/Post_California.csv')
# print(CA_County_Vaccine_Race.shape)
CA_County_Vaccine_Race.head()

,County,Latest_Date,American Indian or Alaska Native (AI/AN),Asian American,Black,Latino,Multi-race,Native Hawaiian or Other Pacific Islander (NHPI),White,Other,Unknown
0,Alameda,2021-04-27,0.002991,0.318446,0.065450,0.122923,0.011829,0.007340,0.310306,0.091716,0.068999
1,Alpine,2021-04-26,0.017435,0.011208,0.002491,0.012453,0.000000,0.000000,0.189290,0.009963,0.757161
2,Amador,2021-04-27,0.008508,0.017541,0.005186,0.079720,0.001224,0.003788,0.743648,0.090326,0.050058
3,Butte,2021-04-27,0.012610,0.041030,0.010404,0.109791,0.012572,0.001887,0.712308,0.061914,0.037485
4,Calaveras,2021-04-27,0.004337,0.017175,0.005193,0.067675,0.000685,0.002454,0.616605,0.078402,0.207475


In [8]:
T = standardize_California(CA_County_Vaccine_Race, US_COUNTY_POPULATION)
state = 'California'

print(T.shape)
T.head()

(58, 18)


,State,County,Geographic Area Name,Date,Rate_AIndA_V,Rate_Asian_V,Rate_Black_V,Rate_Other_V,Rate_White_V,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,California,Alameda,"Alameda, California",2021-04-27,0.002991,0.318446,0.065450,0.091716,0.310306,17797,37797,540391,594001,184368,214392,823593,899377,1671329
1,California,Alpine,"Alpine, California",2021-04-26,0.017435,0.011208,0.002491,0.009963,0.189290,290,310,18,27,4,26,767,813,1129
2,California,Amador,"Amador, California",2021-04-27,0.008508,0.017541,0.005186,0.090326,0.743648,925,1644,665,1043,1066,1374,35636,36844,39752
3,California,Butte,"Butte, California",2021-04-27,0.012610,0.041030,0.010404,0.061914,0.712308,5547,9925,10984,14403,4158,7087,187742,197213,219186
4,California,Calaveras,"Calaveras, California",2021-04-27,0.004337,0.017175,0.005193,0.078402,0.616605,905,1858,859,1399,498,836,41752,43344,45905


In [9]:
not_included = list(set(T['Geographic Area Name'].unique()) - set(CalInfo['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)

    
not_included = list(set(CalInfo['Geographic Area Name'].unique()) - set(T['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


Not included Counties [nan, 'Outside California, Not in California', 'Unknown, Not in California']


In [10]:
T2 = pd.merge(T, CalInfo[['Geographic Area Name', 'cumulative_at_least_one_dose']] , on = 'Geographic Area Name')
print(T2.shape)
T2.head()

(58, 19)


,State,County,Geographic Area Name,Date,Rate_AIndA_V,Rate_Asian_V,Rate_Black_V,Rate_Other_V,Rate_White_V,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,cumulative_at_least_one_dose
0,California,Alameda,"Alameda, California",2021-04-27,0.002991,0.318446,0.065450,0.091716,0.310306,17797,37797,540391,594001,184368,214392,823593,899377,1671329,918683
1,California,Alpine,"Alpine, California",2021-04-26,0.017435,0.011208,0.002491,0.009963,0.189290,290,310,18,27,4,26,767,813,1129,803
2,California,Amador,"Amador, California",2021-04-27,0.008508,0.017541,0.005186,0.090326,0.743648,925,1644,665,1043,1066,1374,35636,36844,39752,17160
3,California,Butte,"Butte, California",2021-04-27,0.012610,0.041030,0.010404,0.061914,0.712308,5547,9925,10984,14403,4158,7087,187742,197213,219186,78431
4,California,Calaveras,"Calaveras, California",2021-04-27,0.004337,0.017175,0.005193,0.078402,0.616605,905,1858,859,1399,498,836,41752,43344,45905,17525


In [11]:
T2.columns

Index(['State', 'County', 'Geographic Area Name', 'Date', 'Rate_AIndA_V',
       'Rate_Asian_V', 'Rate_Black_V', 'Rate_Other_V', 'Rate_White_V',
       'Total_AIndA', 'Total_AIndAC', 'Total_Asian', 'Total_AsianC',
       'Total_Black', 'Total_BlackC', 'Total_White', 'Total_WhiteC',
       'Total_Whole', 'cumulative_at_least_one_dose'],
      dtype='object')

In [12]:
for Race in ['White', 'Black']:
    T2['Numb_' + Race]        = T2['cumulative_at_least_one_dose'] * T2['Rate_' + Race + '_V']
    T2['Rate_' + Race]       = T2['Numb_' + Race] / T2['Total_' + Race]
    # T2['Rate_' + Race + 'C'] = T2['Num_' + Race] / T2['Total_' + Race + 'C']

    
cols = ['State', 'County', 'Geographic Area Name', 'Date', 'Numb_White',
       'Rate_White', 'Numb_Black', 'Rate_Black',
       'Total_AIndA', 'Total_AIndAC', 'Total_Asian', 'Total_AsianC',
       'Total_Black', 'Total_BlackC', 'Total_White', 'Total_WhiteC',
       'Total_Whole', 'cumulative_at_least_one_dose', ]
T = T2[cols]
T.head()

,State,County,Geographic Area Name,Date,Numb_White,Rate_White,Numb_Black,Rate_Black,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,cumulative_at_least_one_dose
0,California,Alameda,"Alameda, California",2021-04-27,285072.846998,0.346133,60127.802350,0.326129,17797,37797,540391,594001,184368,214392,823593,899377,1671329,918683
1,California,Alpine,"Alpine, California",2021-04-26,151.999870,0.198175,2.000273,0.500068,290,310,18,27,4,26,767,813,1129,803
2,California,Amador,"Amador, California",2021-04-27,12760.999680,0.358093,88.991760,0.083482,925,1644,665,1043,1066,1374,35636,36844,39752,17160
3,California,Butte,"Butte, California",2021-04-27,55867.028748,0.297573,815.996124,0.196247,5547,9925,10984,14403,4158,7087,187742,197213,219186,78431
4,California,Calaveras,"Calaveras, California",2021-04-27,10806.002625,0.258814,91.007325,0.182746,905,1858,859,1399,498,836,41752,43344,45905,17525


In [13]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California']


<ipython-input-13-266ad6d67fc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_White,58.0,0.263485,0.102201,0.045942,0.174778,0.271839,0.350576,0.531032
1,Rate_Black,58.0,0.201849,0.093572,0.022966,0.141029,0.193827,0.267941,0.500068
2,Rate_Parity,58.0,0.061636,0.106857,-0.301894,-0.002842,0.042229,0.125973,0.316807


# III. Table States

## [!] B. Tennessee* (Done)


https://www.tn.gov/health/cedep/ncov/data/downloadable-datasets.html

Covid Vaccine County Demographics


95 counties





In [14]:
import pandas as pd

url = 'https://www.tn.gov/content/dam/tn/health/documents/cedep/novel-coronavirus/datasets/COVID_VACCINE_COUNTY_DEMOGRAPHICS.XLSX'
RawData = pd.read_excel(url)
RawData.tail()  # RawData

,DATE,COUNTY,CATEGORY,CAT_DETAIL,RECIPIENT_COUNT
102437,2021-04-28,WILSON,ETHN,UNKNOWN,6041.0
102438,2021-04-28,WILSON,SEX,F,26468.0
102439,2021-04-28,WILSON,SEX,M,20464.0
102440,2021-04-28,WILSON,SEX,O,21.0
102441,2021-04-28,WILSON,SEX,U,82.0


In [15]:
######################################################
Date = Today_Date 
######################################################  


# RawData2 = RawData

while (RawData['DATE'] == Date).sum() == 0:
    print('There is no information for current Date:', Date)
    Date = input('Please Enter a new One:')

    
    

print('We get Tennessee Information for Date:', Date)
RawData = RawData[RawData['DATE'] == Date]

RawData.head()

We get Tennessee Information for Date: 2021-04-28


,DATE,COUNTY,CATEGORY,CAT_DETAIL,RECIPIENT_COUNT
101309,2021-04-28,ANDERSON,RACE,ASIAN,321.0
101310,2021-04-28,ANDERSON,RACE,BLACK OR AFRICAN AMERICAN,600.0
101311,2021-04-28,ANDERSON,RACE,OTHER/MULTIRACIAL,4954.0
101312,2021-04-28,ANDERSON,RACE,UNKNOWN,2686.0
101313,2021-04-28,ANDERSON,RACE,WHITE,21205.0


In [16]:


RawData = RawData[RawData['CATEGORY'].isin(['RACE', 'ETHN'])]


L= []

for county, record in RawData.groupby('COUNTY'):
    # print(county)
    D = {}
    D['County'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        # print(d)
        
        item = d['CAT_DETAIL']
        if item == 'UNKNOWN': 
            item = d['CATEGORY'] + '-' + item
        
        D[item] = d['RECIPIENT_COUNT']
    # print(D)
    L.append(D)

Tennessee_County_Vaccine_Race = pd.DataFrame(L)


In [17]:
# Tennessee_County_Vaccine_Race.to_csv('FinalResult/Tennessee.csv')
print(Tennessee_County_Vaccine_Race.shape)
Tennessee_County_Vaccine_Race.head() # 95 counties in Tennessee

(96, 9)


,County,ASIAN,BLACK OR AFRICAN AMERICAN,OTHER/MULTIRACIAL,RACE-UNKNOWN,WHITE,HISPANIC OR LATINO,NOT HISPANIC OR LATINO,ETHN-UNKNOWN
0,ANDERSON,321.0,600.0,4954.0,2686.0,21205.0,848.0,24924.0,3994.0
1,BEDFORD,116.0,715.0,1544.0,1005.0,8203.0,743.0,9385.0,1455.0
2,BENTON,31.0,205.0,564.0,117.0,4319.0,75.0,4917.0,244.0
3,BLEDSOE,NaN,38.0,356.0,289.0,2455.0,50.0,2620.0,471.0
4,BLOUNT,317.0,737.0,5831.0,3292.0,37418.0,844.0,41340.0,5411.0


In [18]:

# Tennessee_County_Vaccine_Race.to_csv('FinalResult/Tennessee.csv')
print(Tennessee_County_Vaccine_Race.shape)
Tennessee_County_Vaccine_Race.head() # 95 counties in Tennessee

(96, 9)


,County,ASIAN,BLACK OR AFRICAN AMERICAN,OTHER/MULTIRACIAL,RACE-UNKNOWN,WHITE,HISPANIC OR LATINO,NOT HISPANIC OR LATINO,ETHN-UNKNOWN
0,ANDERSON,321.0,600.0,4954.0,2686.0,21205.0,848.0,24924.0,3994.0
1,BEDFORD,116.0,715.0,1544.0,1005.0,8203.0,743.0,9385.0,1455.0
2,BENTON,31.0,205.0,564.0,117.0,4319.0,75.0,4917.0,244.0
3,BLEDSOE,NaN,38.0,356.0,289.0,2455.0,50.0,2620.0,471.0
4,BLOUNT,317.0,737.0,5831.0,3292.0,37418.0,844.0,41340.0,5411.0


In [19]:
# print(len(L))

import numpy as np



state = 'Tennessee' ## 

new_L = []


for idx, row in Tennessee_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        'Dekalb': 'DeKalb',
        'Mcminn': 'McMinn', 
        'Mcnairy': 'McNairy',
        # 'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
        
        
    new_d['Numb_White'] = d['WHITE']
    
    new_d['Numb_Black'] = d['BLACK OR AFRICAN AMERICAN']
    
    
    new_d['Numb_Asian'] = d['ASIAN']
    new_d['Rate_Asian'] = np.nan
    
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

Not included Counties ['Out Of State, Tennessee']
(95, 19)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Tennessee,Anderson,"Anderson, Tennessee",NaN,321.0,600.0,21205.0,0.273891,0.190235,0.300883,371,911,1172,1480,3154,4161,70476,72149,76978
1,Tennessee,Bedford,"Bedford, Tennessee",NaN,116.0,715.0,8203.0,0.223077,0.170279,0.189652,580,843,520,669,4199,4922,43253,44280,49713
2,Tennessee,Benton,"Benton, Tennessee",NaN,31.0,205.0,4319.0,0.316327,0.489260,0.282528,91,220,98,136,419,536,15287,15532,16160
3,Tennessee,Bledsoe,"Bledsoe, Tennessee",NaN,NaN,38.0,2455.0,NaN,0.032759,0.181221,96,188,45,71,1160,1261,13547,13750,15064
4,Tennessee,Blount,"Blount, Tennessee",NaN,317.0,737.0,37418.0,0.243846,0.183333,0.299915,564,1487,1300,1754,4020,5075,124762,126972,133088


In [20]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,90.0,0.210391,0.109482,0.032759,0.153393,0.190545,0.236700,0.771930
1,Rate_White,95.0,0.256798,0.054336,0.139367,0.221444,0.244321,0.286354,0.431321
2,Rate_Parity,90.0,0.049056,0.105565,-0.530581,0.023027,0.063939,0.110407,0.186070


## [!] B. North Carolina* (Done)

Download the Data First

https://covid19.ncdhhs.gov/dashboard/data-behind-dashboards

In [21]:
'CountyVaccine/TMP/April28/North Carolina/Vaccination_Race (3).xlsx'

'CountyVaccine/TMP/April28/North Carolina/Vaccination_Race (3).xlsx'

In [22]:

Date = Today_Date
import os

while True:
    Path = input('Enter North Carolina Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Data = pd.read_excel(Path)
Data.columns



Enter North Carolina Manually collected Data:   CountyVaccine/TMP/April28/North Carolina/Vaccination_Race (3).xlsx


File Exists!


Enter "Yes" if the file contains the information of the date you wanted:   Yes



The Path we have:
CountyVaccine/TMP/April28/North Carolina/Vaccination_Race (3).xlsx


Index(['Index', 'County ', 'Week of', 'American Indian or Alaskan Native',
       'Asian or Pacific Islander', 'Black or African American', 'White',
       'Other', 'Suppressed', 'Missing or Undisclosed'],
      dtype='object')

In [23]:

columns = ['County ', 'Week of', 'American Indian or Alaskan Native',
       'Asian or Pacific Islander', 'Black or African American',
       'Missing or Undisclosed', 'Other', 'Suppressed', 'White']


NorthCarolina = Data[columns]
Groups = NorthCarolina.groupby('County ')

CL = []


for County, df in Groups:
    D = {}
    D['County'] = County
    d = df.sum().to_dict() # Here is a sum
    D['Black or African American'] = d['Black or African American']
    D['American Indian or Alaskan Native'] = d['American Indian or Alaskan Native']
    D['White'] = d['White']
    D['Asian or Pacific Islander'] = d['Asian or Pacific Islander']
    
    CL.append(D)
    
    
NC = pd.DataFrame(CL)
NC = NC.replace(0.0, np.nan) ### Changes here
NC 

,County,Black or African American,American Indian or Alaskan Native,White,Asian or Pacific Islander
0,Alamance,10456.0,199.0,38042.0,1214.0
1,Alexander,NaN,NaN,9252.0,NaN
2,Alleghany,NaN,NaN,2858.0,NaN
3,Anson,2818.0,NaN,NaN,NaN
4,Ashe,NaN,NaN,7902.0,NaN
...,...,...,...,...,...
97,Wayne,8247.0,119.0,16594.0,525.0
98,Wilkes,NaN,NaN,15164.0,NaN
99,Wilson,7366.0,48.0,12494.0,242.0
100,Yadkin,NaN,NaN,9581.0,NaN


In [24]:
# print(len(L))

import numpy as np

state = 'North Carolina' ## 

new_L = []


for idx, row in NC.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mcdowell': 'McDowell'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    
    new_d['Numb_White'] = d['White']
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black or African American']
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Asian or Pacific Islander']
    new_d['Rate_Asian'] = np.nan
    
    new_d['Numb_AIndA'] = d['American Indian or Alaskan Native']
    new_d['Rate_AIndA'] = np.nan
    
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

#     Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
#     Rate = Rate.replace([-np.inf, np.inf], np.nan)
#     T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head()

Not included Counties ['Out Of State, North Carolina', 'Missing, North Carolina']
(100, 21)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_White,Rate_AIndA,Rate_Asian,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,North Carolina,Alamance,"Alamance, North Carolina",NaN,199.0,1214.0,10456.0,38042.0,NaN,0.410412,...,0.305078,2466,3610,2958,3794,35369,37943,124696,128147,169509
1,North Carolina,Alexander,"Alexander, North Carolina",NaN,NaN,NaN,NaN,9252.0,NaN,NaN,...,0.271742,199,376,385,470,2214,2630,34047,34625,37497
2,North Carolina,Alleghany,"Alleghany, North Carolina",NaN,NaN,NaN,NaN,2858.0,NaN,NaN,...,0.268862,59,144,89,107,197,269,10630,10766,11137
3,North Carolina,Anson,"Anson, North Carolina",NaN,NaN,NaN,2818.0,NaN,NaN,NaN,...,NaN,227,368,360,419,11846,12094,11588,11960,24446
4,North Carolina,Ashe,"Ashe, North Carolina",NaN,NaN,NaN,NaN,7902.0,NaN,NaN,...,0.299863,114,269,138,176,262,416,26352,26643,27203


In [25]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,52.0,0.244887,0.046165,0.149604,0.217888,0.241621,0.281076,0.357351
1,Rate_White,91.0,0.294003,0.063943,0.129316,0.250028,0.281583,0.333683,0.477928
2,Rate_Parity,43.0,0.042590,0.045147,-0.041604,0.012808,0.029588,0.076489,0.160193


# IV. OCR States

## [!] C. West Virginia* (Done)


Copy Response from Browsers

https://dhhr.wv.gov/COVID-19/Pages/default.aspx

url = 'https://wabi-us-gov-virginia-api.analysis.usgovcloudapi.net/public/reports/querydata?synchronous=true'




In [26]:
'CountyVaccine/TMP/April28/West Virginia/WestVirginiaApril28.json'

'CountyVaccine/TMP/April28/West Virginia/WestVirginiaApril28.json'

In [27]:


Date = Today_Date
import os

while True:
    Path = input('Enter West Virginia Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Enter West Virginia Manually collected Data:   CountyVaccine/TMP/April28/West Virginia/WestVirginiaApril28.json


File Exists!


Enter "Yes" if the file contains the information of the date you wanted:   Yes



The Path we have:
CountyVaccine/TMP/April28/West Virginia/WestVirginiaApril28.json


In [28]:

import requests
import json


date = Today_Date


with open(Path, 'r') as f:
    response_data = json.load(f)

def get_WestVirginia_Raw(date, response_data):
    

    # data = json.loads(response_data)
    data = response_data

    tmp = data['results'][0]['result']['data']['dsr']['DS'][0]
    # [i for i in tmp]

    tmp2 = tmp['PH'][1]['DM1']
    # len(tmp2)
    # tmp2


    L = []
    for county_info in tmp2:
        d= {}
        d['County'] = county_info['G0']
        d['Date' ] =  date
        try:
            info = [race['C'] for race in county_info['X'][:-1]]

            d['Numb_Black'] = info[0][0]
            d['Numb_White'] = info[-1][0]
            L.append(d)
        except:
            print(d['County'])
            print(county_info['X'])


    WestVirginia = pd.DataFrame(L)
    return WestVirginia
    
    
WestVirginia = get_WestVirginia_Raw(date, response_data)
# WestVirginia.to_csv('FinalResult/Post_WestVirginia.csv')
print(WestVirginia.shape)
WestVirginia.head() # 55 counties


(55, 4)


,County,Date,Numb_Black,Numb_White
0,Barbour,2021-04-28,20,3835
1,Berkeley,2021-04-28,2018,26301
2,Boone,2021-04-28,41,5943
3,Braxton,2021-04-28,14,4349
4,Brooke,2021-04-28,49,6936


In [29]:
# print(len(L))

import numpy as np

state = 'West Virginia' ## 

new_L = []

for idx, row in WestVirginia.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
     'Mcdowell': 'McDowell'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = d['Numb_White']
    # new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Numb_Black']
    
    
    new_d['Date'] = d['Date']
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head()

(55, 19)


,State,County,Geographic Area Name,Date,Numb_Black,Numb_White,Rate_Black,Rate_BlackC,Rate_White,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,West Virginia,Barbour,"Barbour, West Virginia",2021-04-28,20,3835,0.120482,0.071429,0.241788,0.238154,106,225,50,91,166,280,15861,16103,16441
1,West Virginia,Berkeley,"Berkeley, West Virginia",2021-04-28,2018,26301,0.214681,0.170814,0.251713,0.243613,300,1074,1328,2011,9400,11814,104488,107962,119171
2,West Virginia,Boone,"Boone, West Virginia",2021-04-28,41,5943,0.257862,0.157692,0.282127,0.279960,33,95,27,41,159,260,21065,21228,21457
3,West Virginia,Braxton,"Braxton, West Virginia",2021-04-28,14,4349,0.141414,0.078652,0.320156,0.316268,61,135,33,62,99,178,13584,13751,13957
4,West Virginia,Brooke,"Brooke, West Virginia",2021-04-28,49,6936,0.150769,0.094961,0.328176,0.323447,41,148,110,157,325,516,21135,21444,21939


In [30]:
# West Virginia	Barbour	Barbour, West Virginia	2021-03-26	10	2839	0.060241	0.035714	0.178992	0.176303	106	225	50

T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,55.0,0.193326,0.098577,0.017730,0.123082,0.156989,0.280395,0.390805
1,Rate_White,55.0,0.339501,0.055956,0.233949,0.309820,0.332270,0.379104,0.473661
2,Rate_Parity,55.0,0.146175,0.089263,-0.016832,0.092050,0.135448,0.209862,0.329369


# VI. Manual States

## [!] E. Maine* (Done)

In [31]:
'CountyVaccine/TMP/April28/Maine/Part1  - Maine 4_28.csv'

'CountyVaccine/TMP/April28/Maine/Part1  - Maine 4_28.csv'

In [32]:
import os


Date = Today_Date


while True:
    Path = input('Enter Maine Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Enter Maine Manually collected Data:   CountyVaccine/TMP/April28/Maine/Part1  - Maine 4_28.csv


File Exists!


Enter "Yes" if the file contains the information of the date you wanted:   Yes



The Path we have:
CountyVaccine/TMP/April28/Maine/Part1  - Maine 4_28.csv


In [33]:



try:
    Data = pd.read_csv(Path)
except:
    Data = pd.read_excel(Path)
    
Data = Data[Data.columns[1:]]
Maine = Data
# Maine.to_csv('FinalResult/Maine.csv')
Maine = Maine.rename(columns = {'Black or African American': 'Black'})
Maine

,State full,County,Black,White
0,Maine,ANDROSCOGGIN,748,32238
1,Maine,AROOSTOOK,108,20526
2,Maine,CUMBERLAND,2722,127385
3,Maine,FRANKLIN,21,9917
4,Maine,HANCOCK,142,22983
5,Maine,KENNEBEC,298,40489
6,Maine,KNOX,73,16741
7,Maine,LINCOLN,56,16006
8,Maine,OXFORD,88,19390
9,Maine,PENOBSCOT,478,53412


In [34]:
# print(len(L))

import numpy as np
state = 'Maine' ## 

new_L = []
for idx, row in Maine.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    new_d['Numb_White'] = d['White'] if d['White']!= '-' else np.nan
    # new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black'] if d['Black']!= '-' else np.nan
    # new_d['Rate_Black'] = np.nan

    
    # new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

(16, 16)


,State,County,Geographic Area Name,Numb_Black,Numb_White,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Maine,Androscoggin,"Androscoggin, Maine",748,32238,0.153530,0.324610,457,1251,1014,1685,4872,6145,99313,101811,108277
1,Maine,Aroostook,"Aroostook, Maine",108,20526,0.135678,0.322903,1293,1922,343,516,796,1063,63567,64576,67055
2,Maine,Cumberland,"Cumberland, Maine",2722,127385,0.288012,0.469375,1037,2945,7057,9344,9451,11622,271393,277223,295003
3,Maine,Franklin,"Franklin, Maine",21,9917,0.125749,0.339402,146,440,160,259,167,299,29219,29696,30199
4,Maine,Hancock,"Hancock, Maine",142,22983,0.250441,0.436367,261,664,684,878,567,805,52669,53436,54987


In [35]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia', 'Maine']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,16.0,0.218967,0.072847,0.096154,0.152310,0.225312,0.272945,0.323263
1,Rate_White,16.0,0.371856,0.062315,0.267972,0.335704,0.364648,0.416468,0.477520
2,Rate_Parity,16.0,0.152889,0.063363,0.045002,0.115638,0.159058,0.193832,0.262365


## [!]  E. New Jersey* (Done)

In [36]:
'CountyVaccine/TMP/April28/New Jersey/Part1  - NJ April 4_28.csv'

'CountyVaccine/TMP/April28/New Jersey/Part1  - NJ April 4_28.xlsx'

In [37]:
import os

Date = Today_Date


while True:
    Path = input('Enter **New Jersey** Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Enter **New Jersey** Manually collected Data:   CountyVaccine/TMP/April28/New Jersey/Part1  - NJ April 4_28.xlsx


Wrong file path...


Enter **New Jersey** Manually collected Data:   Yes


Wrong file path...


Enter **New Jersey** Manually collected Data:   CountyVaccine/TMP/April28/New Jersey/Part1  - NJ April 4_28.csv


File Exists!


Enter "Yes" if the file contains the information of the date you wanted:   Yes



The Path we have:
CountyVaccine/TMP/April28/New Jersey/Part1  - NJ April 4_28.csv


In [38]:
try:
    Data = pd.read_csv(Path)
except:
    Data = pd.read_excel(Path)
    
if len(Data.columns) == 6:
    Data = Data[Data.columns[1:]]


print(Data.columns)

Data.columns = ['State full', 'County', 'People with at least one vaccine dose','Black', 'White']
NewJersey = Data

# NewJersey.to_csv('FinalResult/NewJersey.csv')

NewJersey.head()

Index(['State full', 'County', 'People with at least one vaccine dose',
       'Black', 'White'],
      dtype='object')


,State full,County,People with at least one vaccine dose,Black,White
0,New Jersey,ATLANTIC,122896,8,58
1,New Jersey,BERGEN,455727,3,51
2,New Jersey,BURLINGTON,210868,6,68
3,New Jersey,CAMDEN,231760,9,62
4,New Jersey,CAPE MAY,48098,2,80


In [39]:
# NewJersey[['Black', 'White'] ].sum(axis=1)

In [40]:
# print(len(L))

import numpy as np

state = 'New Jersey' ## 

new_L = []


for idx, row in NewJersey.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    new_d['Numb_White'] = d['People with at least one vaccine dose'] *  d['White'] / 100
    
    new_d['Numb_Black'] = d['People with at least one vaccine dose'] *  d['Black'] / 100
    
    
    # new_d['Numb_Asian'] = d['First Dose Total'] *  d['Asian %'] / 100
    
    
    # new_d['Numb_Hispa'] = d['First Dose Total'] * d['Hispanic/Latinx %'] / 100
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

(21, 19)


,State,County,Geographic Area Name,Date,Numb_Black,Numb_White,Rate_Black,Rate_BlackC,Rate_White,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,New Jersey,Atlantic,"Atlantic, New Jersey",NaN,9831.68,71279.68,0.217607,0.198387,0.379182,0.367296,1782,3466,21282,23764,45181,49558,187983,194066,263670
1,New Jersey,Bergen,"Bergen, New Jersey",NaN,13671.81,232420.77,0.199498,0.173758,0.342146,0.333198,4508,8451,158020,168160,68531,78683,679302,697546,932202
2,New Jersey,Burlington,"Burlington, New Jersey",NaN,12652.08,143390.24,0.155212,0.140370,0.441900,0.427264,1520,4758,24118,28721,81515,90134,324486,335601,445349
3,New Jersey,Camden,"Camden, New Jersey",NaN,20858.40,143691.20,0.190415,0.176102,0.411090,0.398427,3304,6565,30391,34188,109542,118445,349537,360646,506471
4,New Jersey,Cape May,"Cape May, New Jersey",NaN,961.96,38478.40,0.216025,0.174458,0.455651,0.447054,336,723,943,1366,4453,5514,84447,86071,92039


In [41]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia', 'Maine', 'New Jersey']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,21.0,0.202313,0.047402,0.133585,0.171950,0.195921,0.217607,0.322494
1,Rate_White,21.0,0.357304,0.067753,0.197780,0.318641,0.377106,0.397927,0.455651
2,Rate_Parity,21.0,0.154991,0.059000,0.053449,0.106503,0.144928,0.187642,0.286688


# All in One

## Read Previous Vaccine Data

In [42]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [43]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [44]:
# ######################################

# print(Today_Date)
# ######################################

# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']

    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [45]:
# # state = 'Texas'
# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']

# DF[DF['Rate_Black'] > 0.3][data_cols]

## Save Data

In [46]:

import pickle 

path = 'CountyVaccine/Workplace/STATE_VACCINE_' + Today_Date + '_Manual.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
CountyVaccine/Workplace/STATE_VACCINE_2021-04-28_Manual.p
If yes, please enter: Yes


 Yes


Successfully Saved!


# Merge Data

In [57]:
def merge_data(date):
    D = {}
    for suffix in ['_Manual.p', '_Tableau.p', '_Autonomy.p']:
        path = 'CountyVaccine/Workplace/STATE_VACCINE_' + date + suffix
        print(path)
        with open(path, 'rb') as f:
            s = pickle.load(f)
            
        for k, v in s.items():
            D[k] = v
            
    return D
    
    
date = Today_Date
path = 'STATE_VACCINE_' + date + '.p'
D = merge_data(date)


CountyVaccine/Workplace/STATE_VACCINE_2021-04-28_Manual.p
CountyVaccine/Workplace/STATE_VACCINE_2021-04-28_Tableau.p
CountyVaccine/Workplace/STATE_VACCINE_2021-04-28_Autonomy.p


In [58]:
STATE_VACCINE = D

with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)

print('save data to:', path)


save data to: STATE_VACCINE_2021-04-28.p
